In [1]:
import os
import pylandstats as pls
import geopandas as gpd
import pandas as pd
import time

wd = os.getcwd()

In [2]:
propVars = ["HMM", "disturbance", "fragstats", "nightlights"]

propVars = tuple([s + ".tif" for s in propVars])

In [3]:
d = 'outputs'
parkFolders = [os.path.join(wd, d, o) for o in os.listdir(d) 
                    if os.path.isdir(os.path.join(d,o))]
parkFolders = parkFolders[0:1]
hmmDfs = []
files = []

for park in parkFolders:
    parkName = park.rsplit("\\", 1)[-1]
    #print(parkName)
    subzone = os.path.join(park, parkName + "_PACE_Subzones.shp")
    ppa_gpe = os.path.join(park, parkName + "_PACE.shp")
    #print(subzone)
    rasterLoc = os.path.join(park, "rasters")
    for file in os.listdir(rasterLoc):
        if file.endswith(propVars):
            files.append(file)


In [4]:
dist = os.path.join(rasterLoc, files[0])
frag = os.path.join(rasterLoc, files[1])
hmm = os.path.join(rasterLoc, files[2])
print(dist, "\n", frag, "\n", hmm)

C:\Users\evanmuis.stu\Sync\Masters\Data\outputs\Gar\rasters\Gar_PACE-1985-disturbance.tif 
 C:\Users\evanmuis.stu\Sync\Masters\Data\outputs\Gar\rasters\Gar_PACE-1986-disturbance.tif 
 C:\Users\evanmuis.stu\Sync\Masters\Data\outputs\Gar\rasters\Gar_PACE-1987-disturbance.tif


# function for working with VLCE and disturbance data to get proportions of total area

In [5]:
def propGen(fName, zones):
    metadata = fName.split("\\")[-1].split(".")[0].split("-")
    var = metadata[-1]
    year = metadata[1]

    za = pls.ZonalAnalysis(fName, masks = zones, masks_index_col = 'ppa_gpe')
    class_metrics_df = za.compute_class_metrics_df(metrics = ['proportion_of_landscape', 'total_area'], fillna = False)
    class_metrics_df["park"] = parkName
    class_metrics_df["var"] = var
    class_metrics_df["year"] = year
    return class_metrics_df

# function for working with reclassified VLCE data to get fragstats

In [6]:
def fragGen(fName, zones):
    #obtain metadata for variable and year from filename
    metadata = fName.split("\\")[-1].split(".")[0].split("-")
    var = metadata[-1]
    year = metadata[1]
    
    #create df with subzone information
    df = gpd.read_file(zones)
    newDf = df[["myFID", "ppa_gpe", "ZONE", "SUBZONE", "VARIANT", "PHASE", "NATURAL_DI"]]

    #run pylandstats, adding park, variable, year, and subzone information
    za = pls.ZonalAnalysis(fName, masks = zones, masks_index_col = 'myFID')
    class_metrics_df = za.compute_class_metrics_df(metrics = ['proportion_of_landscape', "total_area", "number_of_patches", "area_mn"], fillna = False)
    class_metrics_df["park"] = parkName
    class_metrics_df["var"] = var
    class_metrics_df["year"] = year
    class_metrics_df = class_metrics_df.reset_index()
    class_metrics_df = pd.merge(class_metrics_df, newDf)
    return class_metrics_df

# creating csv files

In [7]:
d = 'outputs'
parkFolders = [os.path.join(wd, d, o) for o in os.listdir(d) 
                    if os.path.isdir(os.path.join(d,o))]
#parkFolders = parkFolders
hmmDfs = []
distDfs = []
fragDfs = []
fragSubzoneDfs = []
distDfSubzones = []
nightDfs = []
i = 1


for park in parkFolders:
    j = 1
    parkName = park.rsplit("\\", 1)[-1]
    #print(parkName)
    subzone = os.path.join(park, parkName + "_PACE_Subzones.shp")
    ppa_gpe = os.path.join(park, parkName + "_PACE.shp")
    #print(subzone)
    rasterLoc = os.path.join(park, "rasters")
    tifs = []
    for file in os.listdir(rasterLoc):
        if file.endswith(propVars):
            tifs.append(file)
    #print(tifs)
    for file in tifs:
        print(str(i), "/", len(parkFolders), "folders", str(j), "/", len(tifs), "files:", file, end = "\r")
        fileLoc = os.path.join(rasterLoc, file)
        #if file.endswith(propVars[0]): #vlce data
            #hmmDf = propGen(fileLoc, ppa_gpe)
            #hmmDfs.append(hmmDf)
        #if file.endswith(propVars[1]): #disturbance data
            
            #distDf = propGen(fileLoc, ppa_gpe)
            #distDfs.append(distDf)
            
            #distDfSubzone = fragGen(fileLoc, subzone)
            #distDfSubzones.append(distDfSubzone)
            
        if file.endswith(propVars[2]):
            fragDf = propGen(fileLoc, ppa_gpe)
            fragDfs.append(fragDf)
            #fragSubzoneDf = fragGen(fileLoc, subzone)
            #fragSubzoneDfs.append(fragSubzoneDf)
            
        #if file.endswith(propVars[3]):
            #nightDf = propGen(fileLoc, ppa_gpe)
            #nightDfs.append(nightDf)
        j += 1
        #time.sleep(.2)
    i += 1    

#pd.concat(hmmDfs).fillna(0).to_csv("outputCsvs/vlce.csv")
#pd.concat(distDfs).fillna(0).to_csv("outputCsvs/disturbance.csv")
#pd.concat(distDfSubzones).fillna(0).to_csv("outputCsvs/disturbance_subzones.csv")
#pd.concat(fragSubzoneDfs).fillna(0).to_csv("outputCsvs/fragSubzones.csv")
#pd.concat(nightDfs).fillna(0).to_csv("outputCsvs/nightlights.csv")
pd.concat(fragDfs).fillna(0).to_csv("outputCsvs/fragPpaGpe.csv")
print("done")

done6 folders 37 / 37 files: StuartRiverParkLowerSite_PACE-2018-disturbance.tif


In [2]:
raster_filepath = r"C:\Users\evanmuis.stu\Sync\Masters\Data\outputs\Gar\rasters\Gar_PACE-2015-fragstats.tif"
zones_filepath = r"C:\Users\evanmuis.stu\Sync\Masters\Data\outputs\Gar\Gar_PACE_Subzones.shp"

In [34]:
zonalFrag = pls.ZonalAnalysis(raster_filepath, masks=zones_filepath, masks_index_col = "myFID")
pland_df = zonalFrag.compute_class_metrics_df(['proportion_of_landscape', "total_area", "number_of_patches", "area_mn"])
pland_df


metric           proportion_of_landscape  total_area  number_of_patches  \
class_val myFID                                                           
1         1                    23.207301       64.08                3.0   
          2                    13.719886      151.20               98.0   
          3                    11.935093      141.66               91.0   
          4                     6.786753     1585.26             1096.0   
          5                    11.901523      244.08              191.0   
...                                  ...         ...                ...   
3         304                   0.000000        0.00                0.0   
          305                   1.063516      175.41              209.0   
          306                  14.308292     7846.38              522.0   
          307                   0.000000        0.00                0.0   
          308                  10.204082        0.90                4.0   

metric             area_mn  
class_val myFID             
1         1      21.360000  
          2       1.542857  
          3       1.556703  
          4       1.446405  
          5       1.277906  
...                    ...  
3         304     0.000000  
          305     0.839282  
          306    15.031379  
          307     0.000000  
          308     0.225000  

[909 rows x 4 columns]

In [42]:
df = gpd.read_file(zones_filepath)
newDf = df[["ppa_gpe", "myFID", "ZONE", "SUBZONE", "VARIANT", "PHASE", "NATURAL_DI"]]


In [46]:
egg = pland_df.reset_index()

In [47]:
reee = pd.merge(egg, newDf)

In [49]:
reee["park"] = "Gar"

In [50]:
reee

,class_val,myFID,proportion_of_landscape,total_area,number_of_patches,area_mn,ppa_gpe,ZONE,SUBZONE,VARIANT,PHASE,NATURAL_DI,park
0,1,1,23.207301,64.08,3.0,21.360000,PPA,CMA,unp,None,None,NDT5,Gar
1,2,1,76.694915,211.77,8.0,26.471250,PPA,CMA,unp,None,None,NDT5,Gar
2,3,1,0.097784,0.27,3.0,0.090000,PPA,CMA,unp,None,None,NDT5,Gar
3,1,2,13.719886,151.20,98.0,1.542857,PPA,CMA,unp,None,None,NDT5,Gar
4,2,2,59.420171,654.84,90.0,7.276000,PPA,CMA,unp,None,None,NDT5,Gar
...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,2,307,9.580838,1.44,3.0,0.480000,GPE,CWH,dm,None,None,NDT2,Gar
905,3,307,0.000000,0.00,0.0,0.000000,GPE,CWH,dm,None,None,NDT2,Gar
906,1,308,86.734694,7.65,1.0,7.650000,GPE,CWH,dm,None,None,NDT2,Gar
907,2,308,3.061224,0.27,1.0,0.270000,GPE,CWH,dm,None,None,NDT2,Gar


In [14]:
fragGen(fName, zones)

,class_val,myFID,proportion_of_landscape,total_area,number_of_patches,area_mn,park,var,year,ppa_gpe,ZONE,SUBZONE,VARIANT,PHASE,NATURAL_DI
0,1,1,23.207301,64.08,3.0,21.360000,Gar,fragstats,2015,PPA,CMA,unp,None,None,NDT5
1,2,1,76.694915,211.77,8.0,26.471250,Gar,fragstats,2015,PPA,CMA,unp,None,None,NDT5
2,3,1,0.097784,0.27,3.0,0.090000,Gar,fragstats,2015,PPA,CMA,unp,None,None,NDT5
3,1,2,13.719886,151.20,98.0,1.542857,Gar,fragstats,2015,PPA,CMA,unp,None,None,NDT5
4,2,2,59.420171,654.84,90.0,7.276000,Gar,fragstats,2015,PPA,CMA,unp,None,None,NDT5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,2,307,9.580838,1.44,3.0,0.480000,Gar,fragstats,2015,GPE,CWH,dm,None,None,NDT2
905,3,307,NaN,NaN,NaN,NaN,Gar,fragstats,2015,GPE,CWH,dm,None,None,NDT2
906,1,308,86.734694,7.65,1.0,7.650000,Gar,fragstats,2015,GPE,CWH,dm,None,None,NDT2
907,2,308,3.061224,0.27,1.0,0.270000,Gar,fragstats,2015,GPE,CWH,dm,None,None,NDT2


In [8]:
parkName = "Gar"
raster_filepath = r"C:\Users\evanmuis.stu\Sync\Masters\Data\outputs\Gar\rasters\Gar_PACE-2015-fragstats.tif"
zones_filepath = r"C:\Users\evanmuis.stu\Sync\Masters\Data\outputs\Gar\Gar_PACE_Subzones.shp"
fName = raster_filepath
zones = zones_filepath


In [10]:
#obtain metadata for variable and year from filename
metadata = fName.split("\\")[-1].split(".")[0].split("-")
var = metadata[-1]
year = metadata[1]


print(var)
print(year)

fragstats
2015


In [12]:
#create df with subzone information
df = gpd.read_file(zones)
newDf = df[["ppa_gpe", "myFID", "ZONE", "SUBZONE", "VARIANT", "PHASE", "NATURAL_DI"]]
newDf

,ppa_gpe,myFID,ZONE,SUBZONE,VARIANT,PHASE,NATURAL_DI
0,PPA,0,CMA,unp,None,None,NDT5
1,PPA,1,CMA,unp,None,None,NDT5
2,PPA,2,CMA,unp,None,None,NDT5
3,PPA,3,CMA,unp,None,None,NDT5
4,PPA,4,CMA,unp,None,None,NDT5
...,...,...,...,...,...,...,...
304,GPE,304,CWH,dm,None,None,NDT2
305,GPE,305,CWH,dm,None,None,NDT2
306,GPE,306,CWH,dm,None,None,NDT2
307,GPE,307,CWH,dm,None,None,NDT2


In [ ]:
#run pylandstats, adding park, variable, year, and subzone information
za = pls.ZonalAnalysis(fName, masks = zones, masks_index_col = 'myFID')
class_metrics_df = za.compute_class_metrics_df(metrics = ['proportion_of_landscape', "total_area", "number_of_patches", "area_mn"], fillna = False)
class_metrics_df.reset_index()

In [ ]:
class_metrics_df["park"] = parkName
class_metrics_df["var"] = var
class_metrics_df["year"] = year
class_metrics_df.reset_index()
class_metrics_df = pd.merge(class_metrics_df, newDf)